In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import os
from pathlib import Path
import shutil
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
dataset_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ'
label_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/labels/'
data_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/data/'
splitted_dataset_dir = '/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/splitted_dataset/'

In [4]:
if not os.path.exists(splitted_dataset_dir+'train_data'):
    os.makedirs(splitted_dataset_dir+'train_data')

if not os.path.exists(splitted_dataset_dir+'dev_data'):
    os.makedirs(splitted_dataset_dir+'dev_data')
    
if not os.path.exists(splitted_dataset_dir+'test_data'):
    os.makedirs(splitted_dataset_dir+'test_data')

In [5]:
# dev = np.array(pd.read_csv(label_dir+'train_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
# train = np.array(pd.read_csv(label_dir+'test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
# test = np.array(pd.read_csv(label_dir+'dev_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

train = np.array(pd.read_csv(label_dir+'train_split.csv',delimiter=',',encoding='utf-8'))
dev = np.array(pd.read_csv(label_dir+'dev_split.csv',delimiter=',',encoding='utf-8'))
test = np.array(pd.read_csv(label_dir+'test_split.csv',delimiter=',',encoding='utf-8'))

In [6]:
print('train_size: ', len(train))
print('dev_size: ', len(dev))
print('test_size: ', len(test))

train_size:  163
dev_size:  56
test_size:  56


In [9]:
dataset1 = dev
dataset2 = test
dataset3 = train
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))

In [10]:
dataset

array([[300, 'male', 0, 2, 0, 25],
       [301, 'male', 0, 3, 0, 17],
       [306, 'female', 0, 0, 0, 21],
       ...,
       [702, 'male', 0, 0, 0, 19],
       [703, 'male', 0, 8, 0, 28],
       [707, 'male', 0, 1, 0, 23]], dtype=object)

In [11]:
positive = []
negative = []

In [12]:
for i in range(len(dataset)):
    if dataset[i][0]<=310 and dataset[i][0]>=300:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

    elif dataset[i][0]<=355 and dataset[i][0]>=345:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

In [13]:
for i in range(len(dataset)):
        if(dataset[i][2] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))


In [14]:
len(negative)

88

In [15]:
len(positive)

209

In [16]:
def get_data_openface(train_val_test_dir): #data_dir= 'dev_data/'
    participants_IDs_path = {}
    df_openface = pd.DataFrame()
                               
    for name in os.listdir(splitted_dataset_dir + train_val_test_dir):
        if name.startswith('.'):
            continue
        participants_IDs_path[(name[:3])] = splitted_dataset_dir+train_val_test_dir+'/'+name
    
    for key, value in participants_IDs_path.items():
        df_temp = pd.read_csv(value + '/features/' + f'{key}_OpenFace2.1.0_Pose_gaze_AUs.csv')
        indices_to_remove = df_temp[df_temp['success'] == 0].index
        df_temp = df_temp.drop(index=indices_to_remove)
        df_temp.insert(0, 'ID', key)
        df_openface = pd.concat([df_openface, df_temp])
    return df_openface

df_openface_train = get_data_openface('train_data')
df_openface_dev = get_data_openface('dev_data')
df_openface_test = get_data_openface('test_data')

In [17]:
df_openface_train = df_openface_train.sort_values(by=['ID','frame'])
df_openface_dev = df_openface_dev.sort_values(by=['ID','frame'])
df_openface_test = df_openface_test.sort_values(by=['ID','frame'])

In [18]:
df_openface_train = df_openface_train.drop(['frame', 'timestamp', 'confidence', 'success'], axis=1)
df_openface_dev = df_openface_dev.drop(['frame', 'timestamp', 'confidence', 'success'], axis=1)
df_openface_test = df_openface_test.drop(['frame', 'timestamp', 'confidence', 'success'], axis=1)

In [19]:
df_openface_train = df_openface_train.reset_index()
df_openface_train = df_openface_train.drop(['index'], axis=1)

df_openface_test = df_openface_test.reset_index()
df_openface_test = df_openface_test.drop(['index'], axis=1)

df_openface_dev = df_openface_dev.reset_index()
df_openface_dev = df_openface_dev.drop(['index'], axis=1)

In [113]:
df_openface_dev

,ID,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,gaze_0_x,gaze_0_y,gaze_0_z,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,300,69.5,37.7,576.8,0.221,0.036,-0.068,0.015324,0.298824,-0.954185,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,300,69.4,37.4,577.5,0.219,0.037,-0.067,0.006174,0.294520,-0.955626,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,300,69.4,37.3,576.7,0.223,0.039,-0.069,0.005192,0.298043,-0.954538,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,300,69.3,37.3,576.5,0.225,0.039,-0.069,0.005523,0.299662,-0.954030,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,300,69.4,37.3,576.6,0.225,0.039,-0.070,0.005461,0.300506,-0.953764,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1634848,713,13.0,111.1,539.6,0.081,-0.119,0.112,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634849,713,8.7,101.3,500.6,0.122,-0.084,0.093,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634850,713,3.3,92.0,464.5,0.111,-0.041,0.077,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1634851,713,-3.4,87.6,451.4,0.159,-0.009,0.070,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
def scale_down(X):
    X_new = []
    size = 2
    for i in range(int(X.shape[0]/size)):
        cur_row = X[i*size]
        for j in range(1,size):
            if(i+j < X.shape[0]):
                cur_row += X[i+j]
        cur_row = cur_row/size
        X_new.append(cur_row)
    X_new = np.array(X_new)
    return X_new

def decrease_size(X):
    size = 10000
    if(X.shape[0] < size):
        dif = size - X.shape[0] 
        temp = np.zeros((dif,X.shape[1]))
        X = np.concatenate((X,temp),axis = 0)
    elif(X.shape[0] > size):
        X = X[:10000, :]
    return X

def makeDataset(label_path, folder, df_openface):
    df_temp = pd.read_csv(label_path, delimiter=',', encoding='utf-8')
    df_temp = df_temp.drop('Gender', axis=1)
    df_temp = df_temp.drop('PHQ_Score', axis=1)
    df_temp = df_temp.drop('PCL-C (PTSD)', axis=1)
    df_temp = df_temp.drop('PTSD Severity', axis=1)
    file  = np.array(df_temp)     # Participant_ID and PHQ_Binary
    print(file)
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
        patientID = (str(int(file[i][0])))
        df_per_ID = df_openface.loc[df_openface['ID'] == patientID]
        XT = np.array(df_per_ID)
        XT = XT.astype(float)
        XT = scale_down(XT)
        XT = decrease_size(XT)

        X_temp.append(XT)
        Y_temp.append(int(file[i][1]))

    Y_temp = np.asarray(Y_temp)
    return X_temp, Y_temp

In [115]:
X_train, Y_train = makeDataset(label_dir+'train_split.csv', 'train_data', df_openface_train)
X_dev, Y_dev = makeDataset(label_dir+'dev_split.csv', 'dev_data', df_openface_dev)
X_test, Y_test = makeDataset(label_dir+'test_split.csv', 'test_data', df_openface_test)

/Users/misha/Desktop/My_Work/Datasets/DAIC_WOZ/labels/dev_split.csv
dev_data
          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

...

shape3:  (20460, 50)
shape4:  (10000, 50)
patientID:  627
          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze_0_z  ...  AU12_

patientID:  713
          ID  pose_Tx  pose_Ty  pose_Tz  pose_Rx  pose_Ry  pose_Rz  gaze_0_x  \
0        300     69.5     37.7    576.8    0.221    0.036   -0.068  0.015324   
1        300     69.4     37.4    577.5    0.219    0.037   -0.067  0.006174   
2        300     69.4     37.3    576.7    0.223    0.039   -0.069  0.005192   
3        300     69.3     37.3    576.5    0.225    0.039   -0.069  0.005523   
4        300     69.4     37.3    576.6    0.225    0.039   -0.070  0.005461   
...      ...      ...      ...      ...      ...      ...      ...       ...   
1634848  713     13.0    111.1    539.6    0.081   -0.119    0.112  0.000000   
1634849  713      8.7    101.3    500.6    0.122   -0.084    0.093  0.000000   
1634850  713      3.3     92.0    464.5    0.111   -0.041    0.077  0.000000   
1634851  713     -3.4     87.6    451.4    0.159   -0.009    0.070  0.000000   
1634852  713    -35.1    109.0    592.3    0.228   -0.195    0.144  0.223793   

         gaze_0_y  gaze

In [119]:
print(len(X_train))
print(Y_train.shape)

163
(163,)


In [121]:
X_train = np.concatenate((X_train, X_dev), 0)
Y_train = np.concatenate((Y_train, Y_dev), 0)

In [122]:
nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))
print(X_train.shape)
print(Y_train.shape)

(219, 500000)
(219,)


In [123]:
X_test = np.array(X_test)
nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))
print(X_test.shape)

(56, 500000)


In [124]:
clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
clf1.predict(X_test)
print("Random forest: "+ str(clf1.score(X_test,Y_test)))


classifier2 = SVC(kernel = 'rbf', random_state = 0)
classifier2.fit(X_train, Y_train)
classifier2.predict(X_test)
print("SVM rbf: "+ str(classifier2.score(X_test,Y_test)))

clf1.predict(X_train)
print("Random forest: train"+ str(clf1.score(X_train,Y_train)))

classifier2.predict(X_train)
print("SVM: train  "+ str(classifier2.score(X_train,Y_train)))


y_pred = classifier2.predict(X_test)
print(classification_report(Y_test, y_pred))


Random forest: 0.7142857142857143
SVM rbf: 0.6964285714285714
Random forest: train1.0
SVM: train  0.776255707762557
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [138]:
y_pred = clf1.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

Mean squared error: 0.29


In [125]:
y_pred = clf1.predict(X_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      1.00      0.83        39
           1       1.00      0.06      0.11        17

    accuracy                           0.71        56
   macro avg       0.85      0.53      0.47        56
weighted avg       0.80      0.71      0.61        56



In [139]:
y_pred = classifier2.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

Mean squared error: 0.30


In [133]:
# create the model
model = LinearRegression()

# fit the model to the training data
model.fit(X_train, Y_train)

# make predictions on the test data
predictions = model.predict(X_test)

# compute the mean squared error of the predictions
mse = np.mean((predictions - Y_test) ** 2)
print(f'Mean squared error: {mse:.2f}')
mse

Mean squared error: 2211.83


2211.828404798176

In [134]:
Y_train

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [135]:
class RfVideo:
  def __init__(self):
    self.classifier = RandomForestClassifier()
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)

class SVMVideo:
  def __init__(self, kernel = "rbf"):
    self.classifier = SVC(kernel = kernel, random_state = 0)
  
  def fitModel(self, X, Y):
    self.classifier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)

In [136]:
print("Random forest: ")
rfModel = RfVideo()
rfModel.fitModel(X_train, Y_train)
Y_pred1  = rfModel.predictModel(X_test)
print(classification_report(Y_test, Y_pred1))

print("SVM: ")
svmModel = SVMVideo()
svmModel.fitModel(X_train, Y_train)
Y_pred2  = svmModel.predictModel(X_test)
print(classification_report(Y_test, Y_pred2))

Random forest: 


/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56

SVM: 
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        39
           1       0.00      0.00      0.00        17

    accuracy                           0.70        56
   macro avg       0.35      0.50      0.41        56
weighted avg       0.49      0.70      0.57        56



/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misha/My_Projects/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
